In [310]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [311]:
file_path = 'dataset/pilmapres.xlsx'

sheets = ['2021', '2022', '2023']

df_sheets = pd.read_excel(file_path, sheet_name=sheets, dtype={'NIS': str})

In [312]:
df_2021 = df_sheets['2021']
df_2022 = df_sheets['2022']
df_2023 = df_sheets['2023']

df = pd.concat([df_2021, df_2022, df_2023], ignore_index=True)

In [313]:
strings_to_filter = ['External Regional', 'Regional', 'International']

# Create a regex pattern from the list of strings
pattern = '|'.join(strings_to_filter)

In [314]:
df['Level'].value_counts()

External Regional         1010
External National          916
External International     370
Kab/Kota/PT                234
External Provincial         20
Name: Level, dtype: int64

In [315]:
# Exclude Level HERE
df = df[~df['Level'].str.contains('External Regional', na=False)]

# Set Level HERE
# df = df[df['Level'].str.contains('External Regional', na=False)]

In [316]:
df['Major'].value_counts()

Management - Reguler Class              482
Psychology                              191
Communication Science                   180
Accounting                              100
Informatics                              88
Visual Communication Design              80
Information System                       75
Architecture                             67
Fashion Design and Business              49
Medical                                  44
Management - International Class         43
Tourism - Culinary Business              42
Tourism - Hotel and Tourism Business     42
Food Technology Program                  27
Magister of Management                   16
Magister of Management (BUF)              8
Management S3                             6
Name: Major, dtype: int64

In [317]:
# Exclude Major HERE
# df = df[~df['Major'].str.contains('Visual Communication Design', na=False)]

# Set Major HERE
# df = df[df['Major'].str.contains('Visual Communication Design', na=False)]

Data yang dimiliki sudah relatively rapi, tapi untuk membantu processing data, maka data akan di sorting awal

In [318]:
# Sorting sesuai NIS
df_sorted = df.sort_values(by=['NIS', 'Field', 'Score'], ascending=[True, True, False])
df_sorted.head()

,NIS,Name,Major,Student Year,Start Date,End Date,Status,Level,Participant As,Total Participant,Field,Criteria,Score,Period
5,0106012110019,Aldo Clarence Cassius,Management - Reguler Class,2021,2023-09-11,2024-01-07,Satu Tingkat Dibawah Pengurus Harian,Kab/Kota/PT,Individual,NaN,Karir Organisasi,Satu Tingkat Dibawah Pengurus Harian|Kab/Kota/...,2,NaN
6,0106012110019,Aldo Clarence Cassius,Management - Reguler Class,2021,2024-02-19,2024-06-08,Satu Tingkat Dibawah Pengurus Harian,Kab/Kota/PT,Individual,NaN,Karir Organisasi,Satu Tingkat Dibawah Pengurus Harian|Kab/Kota/...,2,NaN
7,0106012110024,Michelle Henny Limbono,Management - Reguler Class,2021,2022-09-25,2022-11-05,Juara 1,External National,Team,50.0,Kompetisi,Juara 1|External National|Team,15,NaN
9,0106012110036,Jesslyn Andressah,Management - Reguler Class,2021,2022-09-24,2023-09-24,Hak Cipta,External National,Team,50.0,Hasil Karya,Hak Cipta|External National|Team,20,NaN
10,0106012110036,Jesslyn Andressah,Management - Reguler Class,2021,2023-04-02,2023-04-02,Hak Cipta,External National,Team,50.0,Hasil Karya,Hak Cipta|External National|Team,20,NaN


Untuk kebutuhan PILMAPRES 2025, telah disediakan grading rubric untuk 7 total bidang di Pedoman PILMAPRES dimana ada beberapa rules yang harus dicatat. <br>
Rule pertama adalah seorang mahasiswa tidak boleh submit more than 4 scores per bidang/field. <br>
Rule kedua adalah seorang mahasiswa tidak boleh submit 10 scores secara total untuk 7 bidang tersebut. <br><br>

Untuk penanganan data mahasiswa yang telah submit more than 4 scores per field ataupun more than 10 secara total, ada beberapa pilihan seperti mengambil 4 scores tertinggi atau terkini per field dan 10 scores tertinggi atau terkini per student. Untuk case ini, akan diambil 4 scores tertinggi per field dan 10 scores tertinggi per student secara total.

In [319]:
# Rule: Max 4 scores per field per student
# Kalau ada yang submit diatas 4 kali di 1 bidang, maka ambil 4 score tertinggi student itu at that specific field
df_sorted = df_sorted.reset_index(drop=True)
df_filtered_field = df_sorted.groupby(['NIS', 'Field'], as_index=False).apply(lambda x: x.nlargest(4, 'Score')).reset_index(drop=True)
df_filtered_field.head()

,NIS,Name,Major,Student Year,Start Date,End Date,Status,Level,Participant As,Total Participant,Field,Criteria,Score,Period
0,0106012110019,Aldo Clarence Cassius,Management - Reguler Class,2021,2023-09-11,2024-01-07,Satu Tingkat Dibawah Pengurus Harian,Kab/Kota/PT,Individual,NaN,Karir Organisasi,Satu Tingkat Dibawah Pengurus Harian|Kab/Kota/...,2,NaN
1,0106012110019,Aldo Clarence Cassius,Management - Reguler Class,2021,2024-02-19,2024-06-08,Satu Tingkat Dibawah Pengurus Harian,Kab/Kota/PT,Individual,NaN,Karir Organisasi,Satu Tingkat Dibawah Pengurus Harian|Kab/Kota/...,2,NaN
2,0106012110024,Michelle Henny Limbono,Management - Reguler Class,2021,2022-09-25,2022-11-05,Juara 1,External National,Team,50.0,Kompetisi,Juara 1|External National|Team,15,NaN
3,0106012110036,Jesslyn Andressah,Management - Reguler Class,2021,2022-09-24,2023-09-24,Hak Cipta,External National,Team,50.0,Hasil Karya,Hak Cipta|External National|Team,20,NaN
4,0106012110036,Jesslyn Andressah,Management - Reguler Class,2021,2023-04-02,2023-04-02,Hak Cipta,External National,Team,50.0,Hasil Karya,Hak Cipta|External National|Team,20,NaN


In [320]:
df_filtered_field['Level'].value_counts()

External National         855
External International    369
Kab/Kota/PT               234
External Provincial        20
Name: Level, dtype: int64

In [321]:
df_filtered_field['Status'].value_counts()

Narasumber/Pembicara                                                                        288
Hak Cipta                                                                                   242
Relawan                                                                                     185
Juara 1                                                                                     174
Juara 2                                                                                     164
Juara 3                                                                                     123
Satu Tingkat Dibawah Pengurus Harian                                                        111
Sekretaris                                                                                   65
Penulis kedua (bukan korespondensi) dst karya ilmiah di journal yg bereputasi dan diakui     51
Wakil Ketua                                                                                  32
Penulis Utama/korespondensi karya ilmiah

In [322]:
df_filtered_field['Major'].value_counts()

Management - Reguler Class              472
Psychology                              172
Communication Science                   163
Accounting                               96
Informatics                              87
Visual Communication Design              80
Information System                       74
Architecture                             67
Fashion Design and Business              49
Medical                                  44
Tourism - Hotel and Tourism Business     42
Tourism - Culinary Business              42
Management - International Class         41
Food Technology Program                  19
Magister of Management                   16
Magister of Management (BUF)              8
Management S3                             6
Name: Major, dtype: int64

In [323]:
# Sum of scores per bidang per student
df_field_totals = df_filtered_field.groupby(['NIS', 'Name', 'Major', 'Field'])['Score'].sum().reset_index()
df_field_totals.head()

,NIS,Name,Major,Field,Score
0,0106012110019,Aldo Clarence Cassius,Management - Reguler Class,Karir Organisasi,4
1,0106012110024,Michelle Henny Limbono,Management - Reguler Class,Kompetisi,15
2,0106012110036,Jesslyn Andressah,Management - Reguler Class,Hasil Karya,40
3,0106012110036,Jesslyn Andressah,Management - Reguler Class,Karir Organisasi,20
4,0106012110040,Ricca Arifa Rachman,Management - Reguler Class,Hasil Karya,0


In [324]:
# Sum of scores per student
df_total_scores_per_student = df_field_totals.groupby(['NIS', 'Name', 'Major'])['Score'].sum().reset_index()
df_total_scores_per_student.head()

,NIS,Name,Major,Score
0,0106012110019,Aldo Clarence Cassius,Management - Reguler Class,4
1,0106012110024,Michelle Henny Limbono,Management - Reguler Class,15
2,0106012110036,Jesslyn Andressah,Management - Reguler Class,60
3,0106012110040,Ricca Arifa Rachman,Management - Reguler Class,0
4,0106012110042,Geovanka Cyndie Tiandya,Management - Reguler Class,40


In [325]:
top_10_students_list = []

# Process each major separately
for major in df_total_scores_per_student['Major'].unique():
    # Filter the dataframe for the current major
    df_major = df_total_scores_per_student[df_total_scores_per_student['Major'] == major]

    # Sort by Score in descending order
    df_major_sorted = df_major.sort_values(by='Score', ascending=False)

    # Take the top 10 students for this major
    df_top_10_major = df_major_sorted.head(10)

    # Append the result to the list
    top_10_students_list.append(df_top_10_major)

In [326]:
df_top_10_all = pd.concat(top_10_students_list)
df_top_10_all.head()

,NIS,Name,Major,Score
86,0106012110357,Naomi Nathanael,Management - Reguler Class,86
102,0106012210022,Febiola,Management - Reguler Class,85
22,0106012110122,Sharon Setiawan,Management - Reguler Class,85
85,0106012110355,Jordan Nathaniel Siswanto,Management - Reguler Class,70
10,0106012110070,Evelin Sutrisno,Management - Reguler Class,65


In [327]:
df_top_10_all = df_top_10_all.sort_values(by=['Major'], ascending=True)
df_top_10_all.head()

,NIS,Name,Major,Score
390,0106042110048,Fabian Benediktus,Accounting,118
398,0106042210008,Nicklaus Stanley,Accounting,125
375,0106042110018,Laurentia Yuke Elsinta,Accounting,95
383,0106042110031,Sean Reynard Wimelson,Accounting,60
371,0106042110007,Vanesa Jocelyn Irtanto,Accounting,60


In [328]:
output_path = 'C:\\Users\\hansj\\OneDrive\\Documents\\Student Affairs\\Projects\\PILMAPRES\\Combined\\output\\output.xlsx'
df_top_10_all.to_excel(output_path, index=False)

In [329]:
import pandas as pd

# Assuming you already have the combined DataFrame from multiple sheets
# Group by NIS, Name, Major, and Field to count the number of submissions per field
df_field_counts = df.groupby(['NIS', 'Name', 'Major', 'Field']).size().reset_index(name='Submissions')

# Now, pivot the data to create a column for each Field showing the number of submissions per student
df_pivot = df_field_counts.pivot_table(index=['NIS', 'Name', 'Major'], 
                                       columns='Field', 
                                       values='Submissions', 
                                       fill_value=0)

# Merge the pivoted DataFrame with the original DataFrame containing the total scores
# Assuming df_total_scores_per_student has the total scores per student
df_final = df_total_scores_per_student.merge(df_pivot, on=['NIS', 'Name', 'Major'], how='left')

# Fill any missing values with 0 for students who don't have submissions in certain fields
df_final.fillna(0, inplace=True)

# Display the final DataFrame
print(df_final.head())

output_path = 'C:\\Users\\hansj\\OneDrive\\Documents\\Student Affairs\\Projects\\PILMAPRES\\Combined\\output\\test.xlsx'
df_final.to_excel(output_path, index=False)

             NIS                     Name                       Major  Score  \
0  0106012110019    Aldo Clarence Cassius  Management - Reguler Class      4   
1  0106012110024   Michelle Henny Limbono  Management - Reguler Class     15   
2  0106012110036        Jesslyn Andressah  Management - Reguler Class     60   
3  0106012110040      Ricca Arifa Rachman  Management - Reguler Class      0   
4  0106012110042  Geovanka Cyndie Tiandya  Management - Reguler Class     40   

   Hasil Karya  Karir Organisasi  Kompetisi  \
0            0                 2          0   
1            0                 0          1   
2            2                 2          0   
3            1                 0          0   
4            2                 0          0   

   Pemberdayaan atau Aksi Kemanusiaan  Pengakuan  
0                                   0          0  
1                                   0          0  
2                                   0          0  
3                               